In [1]:
import mdtraj as md
import numpy as np

In [2]:
# inputdir, outputdir
inputdir = "input"

outputdir = "output"

In [3]:
trj = md.load_trr(f"{inputdir}/out.trr", top=f"{inputdir}/solv_ions_prot.gro")
print(trj)

<mdtraj.Trajectory with 500 frames, 4792 atoms, 309 residues, and unitcells>


In [4]:
trj.xyz.shape

(500, 4792, 3)

# make weight list

In [5]:
from weightdict import atomicWeightsDecimal as wdict
weight_key = 'standard'

atoms_dict = trj.topology.to_dataframe()[0].element

wlist = [float(wdict[atom][weight_key]) for atom in atoms_dict]

# centering

In [6]:
# before centering
for t in range(10):
    tmpxyz = trj.xyz[t]

    total_x = 0
    for i in range(tmpxyz.shape[0]):
        total_x = total_x + tmpxyz[i][0]*wlist[i]
    
    print(total_x)

146172.1851362678
146932.03135434075
148223.58173471055
147453.68239108718
146938.58363073212
146701.03182547682
145840.4915579726
144539.3844968831
144239.59204599247
145374.07823754655


In [7]:
# centering
trj.center_coordinates(mass_weighted=True)

trj.save_trr(f"{outputdir}/test.trr")

In [8]:
# after centering
for t in range(10):
    tmpxyz = trj.xyz[t]

    total_x = 0
    for i in range(tmpxyz.shape[0]):
        total_x = total_x + tmpxyz[i][0]*wlist[i]
    
    print(total_x)

-0.0020182300616653492
-0.0023475031443247474
-0.0022486422465028966
-0.0015107285236837242
-0.0015292740585834963
-0.0015892604226621643
-0.0028540748965362184
-0.002422328964321707
-0.0018043812778678614
-0.0026877619365563987


# choose 2 frames

In [9]:
a = trj.xyz[0]
b = trj.xyz[-1]

# matrix U

In [10]:
U = np.empty((3,3))

for i in range(3):
    for j in range(3):
        U[i][j] = sum( [wlist[n]*a[n ,i]*b[n, j] for n in range(a.shape[0])] )

print(U)

[[ 62952.17991029  38570.72318614 -33253.59856558]
 [ -7879.77936183  37143.47651281  -5878.48978125]
 [ 13068.04244232   8980.31518004  14457.95218838]]


# 固有値問題

In [11]:
# 6*6のomega定義
OMEGA = np.empty((6,6))

for i in range(3):
    for j in range(3):
        OMEGA[i][j] = 0
        OMEGA[i+3][j+3] = 0
        OMEGA[i+3][j] = U[i][j]
        OMEGA[i][j+3] = U.T[i][j]

print(OMEGA)

[[     0.              0.              0.          62952.17991029
   -7879.77936183  13068.04244232]
 [     0.              0.              0.          38570.72318614
   37143.47651281   8980.31518004]
 [     0.              0.              0.         -33253.59856558
   -5878.48978125  14457.95218838]
 [ 62952.17991029  38570.72318614 -33253.59856558      0.
       0.              0.        ]
 [ -7879.77936183  37143.47651281  -5878.48978125      0.
       0.              0.        ]
 [ 13068.04244232   8980.31518004  14457.95218838      0.
       0.              0.        ]]


In [12]:
# 固有値問題
eig_val, eig_vec =np.linalg.eig(OMEGA)

# pairs
for i in range(len(eig_val)):
    print(eig_val[i])
    print(eig_vec[i])
    print('----------')

82912.57364980309
[ 0.52024659  0.52024659  0.41870069  0.23245047 -0.41870069 -0.23245047]
----------
-82912.57364980296
[ 0.39369797  0.39369797 -0.56934363  0.1443944   0.56934363 -0.1443944 ]
----------
35148.67726125713
[-0.27266352 -0.27266352 -0.02318544  0.65201     0.02318544 -0.65201   ]
----------
19577.861617531405
[ 0.68750629 -0.68750629  0.14706458 -0.07554547  0.14706458 -0.07554547]
----------
-35148.677261257166
[ 0.14625927 -0.14625927 -0.6916441  -0.01538383 -0.6916441  -0.01538383]
----------
-19577.86161753139
[ 7.70930131e-02 -7.70930131e-02  6.68567349e-04  7.02891329e-01
  6.68567349e-04  7.02891329e-01]
----------


In [13]:
# root2を出す
hks = eig_vec * np.sqrt(2)

h = hks[:, 0:3]
k = hks[:, 3:6]

print(h.shape, k.shape)

(6, 3) (6, 3)


# 回転行列R

In [14]:
R = np.dot(h.T,k)

In [22]:
print(R)
print(np.linalg.det(R))

[[ 2.29998472e-16 -9.87117497e-17 -1.29050539e-16]
 [-7.85838027e-17 -3.32603894e-16 -2.48461235e-17]
 [-6.79778689e-16 -5.67086218e-16  1.41141500e-17]]
1.7329885989955432e-47


# bをrへ変換

In [16]:
r = np.empty_like(b)

for i in range(3):
    for n in range(r.shape[0]):
        r[n,i] = sum( [R[i,j]*b[n,j] for j in range(3)] )

# save

In [17]:
fitted_xyz = np.array([a,r])

fitted_xyz.shape

(2, 4792, 3)

In [18]:
fitted_trj = md.Trajectory(fitted_xyz, trj.topology)

In [19]:
fitted_trj.save_trr(f"{outputdir}/test.trr")